# 中文篇章级句间关系分类 - LoRA微调实验

## 实验说明

本实验使用 **MindSpore + mindnlp 0.5.1 + LoRA** 在 **DeepSeek-R1-Distill-Qwen-1.5B** 模型上进行微调。

### 任务目标
- **输入**：一个句子或对话片段
- **输出**：该句子所属的PDTB篇章关系分类（扩展/因果/比较/并列/其他）以及分类原因

### 版本信息
- mindnlp: 0.5.1
- mindspore: 2.7.0
- transformers: ~4.40-4.45（推荐）
- 数据类型: bfloat16（在Ascend NPU上更稳定）

### ⚠️ 重要注意事项
1. **不要使用 `fp16=True`**：会导致 `mindtorch.npu.amp` 错误
2. **不要使用 `gradient_checkpointing=True`**：可能导致兼容性问题
3. **使用 `bfloat16` 数据类型**：在Ascend NPU上表现更稳定
4. **数据路径**：确保数据已上传至 `/home/ma-user/work/data/`

### 训练环境
- 镜像：mindspore_2_7-vllm-mindspeed-cann8_2alpha2_ubuntu22
- 实例规格：Ascend: 1*ascend-snt9b1 | ARM: 24核 192GB

## 1. 导入必要的库

In [ ]:
# 核心框架
import mindnlp
import mindspore
from mindnlp import core

# 数据处理
from datasets import Dataset
import pandas as pd

# 模型和训练相关
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    DataCollatorForSeq2Seq, 
    TrainingArguments, 
    Trainer
)

# LoRA相关
from peft import LoraConfig, TaskType, get_peft_model

# 查看版本信息
print(f"mindnlp版本: {mindnlp.__version__}")
print(f"mindspore版本: {mindspore.__version__}")

## 2. 加载数据集

数据集格式：
```json
{
    "content": "输入的句子或对话内容",
    "summary": "关系分类结果\n原因：详细解释"
}
```

In [ ]:
# 数据路径（华为云ModelArts路径）
train_path = "/home/ma-user/work/data/train.json"
val_path = "/home/ma-user/work/data/val.json"

# 读取数据
df_train = pd.read_json(train_path)
df_val = pd.read_json(val_path)

# 转换为Dataset格式
ds_train = Dataset.from_pandas(df_train)
ds_val = Dataset.from_pandas(df_val)

# 查看数据集信息
print(f"训练集样本数: {len(ds_train)}")
print(f"验证集样本数: {len(ds_val)}")
print("\n数据集前3个样本:")
ds_train[:3]

## 3. 加载Tokenizer

使用 DeepSeek-R1-Distill-Qwen-1.5B 的分词器

In [ ]:
# 加载tokenizer
model_name = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    use_fast=False, 
    trust_remote_code=True
)

# 查看tokenizer信息
print(f"词汇表大小: {tokenizer.vocab_size}")
print(f"最大长度: {tokenizer.model_max_length}")
print(f"PAD token: {tokenizer.pad_token}")
print(f"EOS token: {tokenizer.eos_token}")
tokenizer

## 4. 数据预处理

将原始数据转换为模型可接受的格式：
- 使用对话模板格式
- 设置最大长度为384
- 只对回答部分计算损失（instruction部分设为-100）

In [ ]:
# 最大序列长度
MAX_LENGTH = 384

def process_func(example):
    """
    数据处理函数
    
    将数据转换为对话格式：
    <|im_start|>system
    你是PDTB文本关系分析助手<|im_end|>
    <|im_start|>user
    {用户输入}<|im_end|>
    <|im_start|>assistant
    {模型回答}<|im_end|>
    """
    # 构建指令部分（system + user）
    instruction = tokenizer(
        f"<|im_start|>system\n你是PDTB文本关系分析助手<|im_end|>\n"
        f"<|im_start|>user\n{example.get('content', '') + example.get('input', '')}<|im_end|>\n"
        f"<|im_start|>assistant\n",
        add_special_tokens=False
    )
    
    # 构建回答部分
    response = tokenizer(
        f"{example.get('summary', '')}", 
        add_special_tokens=False
    )

    # 拼接input_ids和attention_mask
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    
    # 构建labels：指令部分设为-100（不计算损失），只对回答部分计算损失
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]

    # 截断到最大长度
    input_ids = input_ids[:MAX_LENGTH]
    attention_mask = attention_mask[:MAX_LENGTH]
    labels = labels[:MAX_LENGTH]

    return {
        "input_ids": input_ids, 
        "attention_mask": attention_mask, 
        "labels": labels
    }

# 处理训练集和验证集
print("开始处理训练集...")
tokenized_train = ds_train.map(process_func, remove_columns=ds_train.column_names)
print("训练集处理完成！")

print("\n开始处理验证集...")
tokenized_val = ds_val.map(process_func, remove_columns=ds_val.column_names)
print("验证集处理完成！")

# 查看处理后的第一个样本
print("\n处理后的第一个样本解码结果:")
print(tokenizer.decode(tokenized_train[0]['input_ids']))

## 5. 加载基础模型

### 关键配置说明
- `ms_dtype=mindspore.bfloat16`: 使用bfloat16数据类型，在Ascend NPU上更稳定
- `device_map=0`: 指定设备映射
- `enable_input_require_grads()`: 开启输入梯度计算（LoRA训练必需）

In [ ]:
# 加载基础模型
print("正在加载模型，请稍候...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    ms_dtype=mindspore.bfloat16,  # 使用bfloat16数据类型
    device_map=0  # 指定设备
)

# 开启梯度计算（LoRA训练必需）
model.enable_input_require_grads()

print("模型加载完成！")
print(f"模型参数量: {model.num_parameters():,}")

## 6. 配置LoRA

### LoRA参数说明
- `task_type`: 任务类型（CAUSAL_LM表示因果语言模型）
- `target_modules`: 要应用LoRA的模块（注意力层和FFN层）
- `r=8`: LoRA秩，控制参数量
- `lora_alpha=32`: LoRA缩放因子，通常设为r的4倍
- `lora_dropout=0.1`: Dropout率
- `inference_mode=False`: 训练模式

In [ ]:
# 配置LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # 注意力层
        "gate_proj", "up_proj", "down_proj"       # FFN层
    ],
    r=8,                    # LoRA秩
    lora_alpha=32,          # LoRA缩放因子
    lora_dropout=0.1,       # Dropout率
    inference_mode=False    # 训练模式
)

# 应用LoRA到模型
model = get_peft_model(model, lora_config)

# 打印可训练参数信息
model.print_trainable_parameters()

## 7. 配置训练参数

### 训练参数说明
- `output_dir`: 输出目录
- `per_device_train_batch_size=4`: 每个设备的batch size
- `gradient_accumulation_steps=5`: 梯度累积步数（有效batch size = 4 × 5 = 20）
- `num_train_epochs=3`: 训练轮数
- `learning_rate=3e-5`: 学习率
- `logging_steps=10`: 每10步记录一次日志
- `save_steps=100`: 每100步保存一次checkpoint

### ⚠️ 注意
- **不要添加 `fp16=True`**：会导致NPU兼容性错误
- **不要添加 `gradient_checkpointing=True`**：可能导致兼容性问题

In [ ]:
# 定义训练参数
args = TrainingArguments(
    output_dir="./output",                    # 输出目录
    per_device_train_batch_size=4,            # batch size
    gradient_accumulation_steps=5,            # 梯度累积步数
    logging_steps=10,                         # 日志记录间隔
    num_train_epochs=3,                       # 训练轮数
    save_steps=100,                           # checkpoint保存间隔
    learning_rate=3e-5,                       # 学习率
    save_on_each_node=True,                   # 在每个节点上保存
    # 注意：不要添加fp16=True或gradient_checkpointing=True
)

print("训练参数配置完成！")
print(f"有效batch size: {args.per_device_train_batch_size * args.gradient_accumulation_steps}")
print(f"总训练步数: {len(tokenized_train) // (args.per_device_train_batch_size * args.gradient_accumulation_steps) * args.num_train_epochs}")

## 8. 创建Trainer并开始训练

In [ ]:
# 创建Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

print("Trainer创建成功！")

In [ ]:
# 开始训练
print("========== 开始训练 ==========")
print("预计训练时间: 约1.5-2小时（基于之前的训练经验）")
print("="*50)

trainer.train()

## 9. 训练总结

训练完成后：
- LoRA权重保存在 `./output/checkpoint-xxxx/` 目录中
- 主要文件包括：
  - `adapter_model.safetensors`: LoRA适配器权重
  - `adapter_config.json`: LoRA配置
  - `trainer_state.json`: 训练状态

### 下一步
使用 `merge.ipynb` 脚本将LoRA权重与基础模型合并，或直接使用checkpoint进行推理。

In [ ]:
# 查看训练结果文件
import os
print("训练输出目录内容:")
if os.path.exists('./output'):
    for item in os.listdir('./output'):
        print(f"  - {item}")
else:
    print("输出目录尚未创建")